# How to Train your Perceptron-1
**An opinionated recipe to get the most performance on your first few tries**

For a more traditional recipe check out [A Recipe for Training Neural Networks](http://karpathy.github.io/2019/04/25/recipe/) by [Andrej Karpathy](https://karpathy.ai/)

### A perceptron (a.k.a. artificial neuron) is the building block of a traditional neural network (a.k.a Multilayer Perceptron; MLP)


<img src="data/img/neural_net.png">

([image source](https://pubmed.ncbi.nlm.nih.gov/28087243/))

In astronomy, we encounter mainly these three data types:
- **Catalogs (i.e. tabulated numerical data)** &rarr; This Notebook
- Images (i.e. data with spatial dependence/ evolution)
- Time Series (i.e. data with temporal dependence/evolution)

and we mainly encounter two types of tasks:
- Regression (i.e. predict a continuous value)
- Classification (i.e. predict a discrete value)

**Basic Training algorithm:**
- Define a model (a sequence of matrix multiplication)
- Start with an initial guess for all the parameters
- Find the derivative of loss function with respect to parameters
- modify the parameters by some step to reduce the derivate value.
- take multiple steps until derivative is zero (ideally) or loss minima is reached.

<img src="data/img/gradient_descent_animation.gif">

([image source](https://towardsdatascience.com/gradient-descent-animation-1-simple-linear-regression-e49315b24672#:~:text=The%20Gradient%20Descent%20method%20is,and%20costs%20during%20gradient%20descent.))

### Your first neural Network:
Which Python framework to use?

There are two main options, [Tensorflow](https://www.tensorflow.org/) and [Pytorch](https://pytorch.org/). Both give you the exact same functionalities.

- In a hurry? &rarr; Use Tensorflow
- Long research project? &rarr; Use Pytorch

Today we are in a hurry!

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers

We will take the case of photometric redshift estimation using magnitudes and colors as our example. This is a regression task. We will deal with classification exclusively in a future tutorial but will spread some tidbits for classification along the way. Let's read in the data.

In [ ]:
base_path = Path("./data/photoz_catalogues/Teddy")
teddy_A = pd.read_csv(
    base_path / "teddy_A",
    delim_whitespace=True,
    comment="#",
    names=[
        "id",
        "mag_r",
        "u-g",
        "g-r",
        "r-i",
        "i-z",
        "z_spec",
        "feat1",
        "feat2",
        "feat3",
        "feat4",
        "feat5",
    ],
)

### Select features carefully and ALWAYS scale the data

**NOTE:**
Feature = predictors = independent variable = input | Target = output = dependent variable

Neural Networks are not the best at automatic feature selection, so better to start with a small number of physically motivated features which are (hopefully) independent. 

Feature scaling takes away the effect of Units and does not let a feature dominate. There are many ways to scale, subtracting each feature's mean and dividing by their standard deviation aka Standard Scaling is a good start.

In [ ]:
X = teddy_A[["mag_r", "u-g", "g-r", "r-i", "i-z",]]
Y = teddy_A["z_spec"].values

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

## For regression: If the target variable is bounded scale it to an unbounded space using a sigmoidal function

Bounded= values are either floored or capped or both


One such function that works well is:
$$h(x) = \log \left( \dfrac{x-x_{\mathrm{min}}}{x_{\mathrm{max}}-x}\right)$$



In [ ]:
def logistic_transform(x, xmin, xmax):

    return np.log((x - xmin) / (xmax - x))


def inverse_logistic_transform(x, xmin, xmax):

    return (xmax * np.exp(x) + xmin) / (1 + np.exp(x))

In [ ]:
Y = logistic_transform(Y, 0, 1)

### Set a random seed
Much of the following will be probabilistic in nature. For the sake of reproducibility during development se a random seed.

In [ ]:
SEED = 42
tf.random.set_seed(SEED)

### Randomly divide available data into 3 sets: Training, validation, test

There is a fourth data set which is not available to us at this moment. This will be the one used to evaluate the method when everything has been finalised. If the data set is large, 80:10:10 is a good split.

In [ ]:
n_gal = len(X)
frac_train = 0.8
frac_val = 0.1

rng = np.random.default_rng(SEED)
indices = rng.permutation(n_gal)

ind_split_train = int(np.ceil(frac_train * n_gal))
ind_split_val = ind_split_train + int(np.ceil(frac_val * n_gal))

In [ ]:
x_train = X[indices[:ind_split_train]]
x_val = X[indices[ind_split_train:ind_split_val]]
x_test = X[indices[ind_split_val:]]

y_train = Y[indices[:ind_split_train]]
y_val = Y[indices[ind_split_train:ind_split_val]]
y_test = Y[indices[ind_split_val:]]

Choosing MLP architecture:
- Start small and increase model size untill performance saturates for a fixed amount of data
- Wider networks are good at memorization whereas deep networks are good at generalization
- Generally a wide first- narrow later structure works well regression


Choosing activation function: (See [this](https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-prelu-with-keras.md) for reasoning)
- Always use the "leaky reLu" family for hidden layers
- Use linear activation for the output layer for regression and sigmoid/softmax for classification

Choosing an initializer for weights:
- For reLu family of activations use He-normal
- For Sigmoid/Tanh family of activations use Glorot-Uniform

See [this](https://www.deeplearning.ai/ai-notes/initialization/) to see why initialization is important. 

In [ ]:
model_input = layers.Input(shape=(5,))
h1 = layers.Dense(100, kernel_initializer="he_normal")(model_input)
o1 = layers.PReLU()(h1)
h2 = layers.Dense(50, kernel_initializer="he_normal")(o1)
o2 = layers.PReLU()(h2)
output = layers.Dense(1, activation="linear")(o2)
model = tf.keras.models.Model(inputs=model_input, outputs=output)
# summarize layers
print(model.summary())

**Choosing an optimizer:**
- Just use Adam with default learning rate

**Choosing a loss**
- Mean squared error (MSE) for regression and cross entropy for classification. They maximize $p(\mathrm{data}|\mathrm{parameters})$ under the assumption of a gaussian data generating process and are good starting points.
- Classification accuracy is a good metric for classification. MSE is a good metric for regression.

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=tf.keras.losses.MeanSquaredError(),
)

**Fitting the model**
- Batch size: The general norm is to maximize it until you run out of memory. This is a good recipe for image data but not for catalogs, where often the whole data set can fit in memory. In that case the network will see only one batch and overfit to the training data.
- Shuffle the data during each epoch. 

In [ ]:
n_epochs = 100
batch_size = 500

history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=batch_size,
    epochs=n_epochs,
    verbose=1,
    callbacks=None,
    validation_data=(x_val, y_val),
    shuffle=True,
)

**How many epochs do we let it train for?**


Plot the train and validation loss as a function of epoch
- If everything is right, training loss should decrease with increasing epoch and validation loss decreases reaches a minima and then increases.
- We select the model with the minimum validation loss

In [ ]:
epoch = np.arange(1, n_epochs+1)

plt.plot(epoch, history.history["loss"], label="Training loss")
plt.plot(epoch, history.history["val_loss"], label="Validation loss")
plt.legend()
plt.show()

**If performance above looks good, we are done! Else:**
- Learning rate decay
- Change the model architecture